# Wines

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib as plt
import scipy 
from scipy import stats
import folium

In [2]:
df = pd.read_csv('winemag-data-130k-v2.csv')
pd.set_option("display.max_columns", 30) 
pd.set_option("display.max_rows", 120)
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


## Missing data

In [ ]:
df.isnull().sum()

In [ ]:
# excluding columns that will not be used
df = df.drop(['Unnamed: 0','taster_twitter_handle','region_2'], axis='columns')

## Categorical variables

In [ ]:
# checking a full description 
df['description'][0]

In [ ]:
# finding wines from Portugal
portugal_wn = df.loc[df.country == 'Portugal']
portugal_wn.head()

## Numerical variables

In [ ]:
max_points = df.points.max()
min_points = df.points.min()
mean_points = df.points.mean().round(2)
median_points = df.points.median().round(2)
general_points = pd.DataFrame({'Max':[max_points],
                                'Min':[min_points],
                                'Mean':[mean_points],
                                'Median':[median_points]},
                                index=['Values'])
general_points


In [ ]:
px.histogram(df, 'points', nbins=30, title='Histogram of points')

In [ ]:
# testing if the mean of points from Portugal is the same general mean of 88.45 points
general_mean = mean_points
mean_portugal = df.loc[df.country == 'Portugal',:]
points_portugal = mean_portugal.points

In [ ]:
# hypothesis - One simple T-test
# H0 (null): points_mean_portugal = 88.45
# H1(alternative): points_mean_portugal != 88.45

In [ ]:
# test
tStat, pValue =  scipy.stats.ttest_1samp(points_portugal, general_mean, axis=0)

print("P-Value:{0} \t T-Statistic:{1}".format(pValue,tStat))

##### As p-value < 0.05 we have less a than 5% chance the results will be random, so we reject the H0 and Portugal does not have the average of points equal to the general average.

In [ ]:
max_price = df.price.max()
min_price = df.price.min()
mean_price = df.price.mean().round(2)
median_price = df.price.median().round(2)
general_prices = pd.DataFrame({'Max':[max_price],
                                'Min':[min_price],
                                'Mean':[mean_price],
                                'Median':[median_price]},
                                index=['Values'])
general_prices

In [ ]:
px.histogram(df, 'price', nbins=200, title='Histogram of prices')

## Looking for outliers

In [ ]:
px.box(df, 'points', orientation='h')

In [ ]:
px.box(df, 'price', orientation='h')

In [ ]:
px.box(df.loc[df['price'] < 100, :], 'price', orientation='h')

In [ ]:
px.box(df.loc[(df['price'] > 20) & (df.price < 100), :], 'price', orientation='h')

## Treating missing data

In [ ]:
# replacing missing data
df.fillna({'country':'Unkwown',
            'designation':'Unkwown',
            'province':'Unknown',
            'region_1':'Unknown',
            'taster_name':'Unknown',
            'price':df.price.median()}, inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.sample(5)

In [ ]:
# checking characteristics of Nan in Variety
df[df.variety.isna()]

In [ ]:
# looking for more characteristics by index
df['description'][86909]

In [ ]:
# filtering the dataset by the same characteristics as Nan in Variety
carmen = df[df.country == 'Chile'][df.points == 88][df.price == 17][df.province == "Maipo Valley"]
carmen

In [ ]:
# mapping keywords in the description of 'Carmen'
carmen.description.map(lambda desc: "blackberry" in desc)

In [ ]:
# description by index of 'Carmen' 
df['description'][89627]

In [ ]:
# replacing the Nan variety with 'Cabernet Sauvignon' - this was chosen based on the same characteristics
df.variety.fillna("Cabernet Sauvignon", inplace=True)

## General inspection

In [ ]:
# counts how many times there are 'tropical' and 'fruity' in the dataset
trop = df.description.map(lambda desc: "tropical" in desc).sum()
fruit = df.description.map(lambda desc: "fruity" in desc).sum()
descritpt_counts = pd.Series([trop, fruit], index=['tropical', 'fruity'])
descritpt_counts

In [ ]:
# creating a ranking by points
def stars(linha):
    if linha.points >= 95:
        return 3
    elif linha.points >= 85:
        return 2
    else:
        return 1
all_stars = df.apply(stars, axis='columns')

In [ ]:
df['stars'] = all_stars
df.head()

In [ ]:
px.histogram(df, 'stars', nbins=5)

In [ ]:
# maior pontuação e maior preço de cada país
melhores_pais = pd.pivot_table(df, values=['points','price'], 
                                index=['country'], aggfunc=np.max).reset_index().sort_values(ascending=False,by='points')
melhores_pais

In [ ]:
#agrupando os preços por pontos máx e min
df.groupby('price').points.agg([max,min]).reset_index()

In [ ]:
#agrupando variedade por preço máx e mín
variedades_preco = df.groupby('variety').price.agg([max,min]).reset_index().sort_values(by=['max','min'], ascending=False)
variedades_preco

In [ ]:
# agrupando quantidade de Variedade por País
df.groupby(['country','variety']).size().sort_values(ascending=False)

### What is the best score(points) for each country? Which countries have the best wine?

In [ ]:
country_points = pd.pivot_table(df, values=['points'], 
                                index=['country'], 
                                aggfunc=np.max).reset_index().sort_values(ascending=False,by='points')
country_points

In [ ]:
px.bar(country_points, x='country', y='points', text_auto='.2s',  title='Maximum points for each country', labels={'points':'Points', 'country':'Country'})

### Where are the best wines regionally located, their variety and price?

In [ ]:
best_region = pd.pivot_table(df.loc[df.points == 100,:], values=['price'], 
                                index=['province', 'region_1', 'variety'], aggfunc=np.min).reset_index().sort_values(ascending=False,by='price')

best_region

###  How many are there wine varieties?

In [ ]:
df.variety.nunique()

### Which Taster rated the most wines?

In [ ]:
df.taster_name.value_counts()

In [ ]:
df.loc[df.taster_name == 'Roger Voss', ['country', 'variety','points', 'price']]

In [ ]:
# mean of the points of each Taster
taster = df.groupby('taster_name').points.mean().sort_values(ascending=False)
taster

### What is the most expensive wine in each country?

In [ ]:
pd.pivot_table(df, values=['price'], 
                        index=['country'], aggfunc=np.max).reset_index().sort_values(ascending=False,by='price')

### What is the average price of wines in each country?

In [ ]:
mean_price = pd.pivot_table(df, values=['price'], index=['country'], aggfunc=np.mean).reset_index()

In [ ]:
px.bar(mean_price, x='country', y='price', text_auto='.2s', title='Average wine price in each country', labels={'price':'Price', 'country':'Country'})

### Where are the cheapest wines?

In [ ]:
lowest_price = pd.pivot_table(df.loc[df.price <= 4], values=['price'], 
                                                     index=['country'], 
                                                     aggfunc=np.min).reset_index().sort_values(ascending=True, by='price')
lowest_price

### Which wine has the highest cost-benefit ratio between price and points?

In [ ]:
# highest score and lowest price ratio
ratio_idx = (df.points / df.price).idxmax()
ratio = df.loc[ratio_idx]
ratio

### What is the standard deviation of price and points?

In [ ]:
std_price = df.price.std().round(2)
std_points = df.points.std().round(2)
print('Standard deviation of price:', std_price)
print('Standard deviation of points:', std_points)

### What is the most expensive and cheapest type of wine?

In [ ]:
df.price.max()

In [ ]:
more_expensive = df.loc[df.price == df.price.max()]['variety']
more_expensive

In [ ]:
df.price.min()

In [ ]:
cheaper = list((df.loc[df.price == 4]['variety']).unique())
cheaper

# Maps

In [ ]:
# reading a file to get the latitude and longitude data
dc = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv')
dc.head()

In [ ]:
dc = dc.drop(['country_code','usa_state_code','usa_state_latitude','usa_state_longitude','usa_state'], axis='columns')
dc.sample(10)

In [ ]:
df1 = pd.merge(df, dc, how = 'inner', on = 'country')

In [ ]:
# putting the data on a map
cols = ['country', 'points','variety','latitude', 'longitude']
data_plot = df1.loc[:, cols].sample(100)

In [ ]:
# creating a new column
data_plot.loc[:,'color'] = 'NA'
data_plot.loc[data_plot['variety'] == 'Bordeaux-style Red Blend', 'color'] = 'blue'


In [ ]:
# drawing a map
map = folium.Map()

# putting the best wines on the map
for index, location_info in data_plot.iterrows():
		folium.Marker( [location_info['latitude'], location_info['longitude']],
										popup = location_info['country'],
										icon = folium.Icon(color=location_info['color']) ).add_to(map)

map
